<a href="https://colab.research.google.com/github/murffps/machine-learning-titanic/blob/main/MLBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### Project Introduction ########
Titanic Data Prediction with Classification Supervised Learning : 

*   Prediction label for an individual as "survived" or "died"

*    take passenger/trip information and predict whether that passenger would survive:  

*   Is this a good ML question for good data for ML ? Why ? Why not? initial assumptions.
how to properly document this 
Cost Time trade offs  princples estimate
Business value ROI 
Success how to measure given the above https://github.com/mattharrison/ml_pocket_reference/blob/master/ch03.ipynb 


Algebreic Representation of Intent to create a function 
 y = f(x) to turn a feature into a label 
 x is a matrix, every column in x is a feature 
 the output y is a vector that contains labels(or targets) (classification uses labels or regression would use values) 

 



ML WORKFLOW STEPS 
Gather
clean 
Create feautres
normlaize 

sample data
testing data
training data
create model 
evaluate model - possible change question
deploy model

The CRoss Industry Standard Process for Data Mining (CRISP-DM) is a process model that serves as the base for a data science process. It has six sequential phases:

Business understanding – What does the business need?
Data understanding – What data do we have / need? Is it clean?
Data preparation – How do we organize the data for modeling?
Modeling – What modeling techniques should we apply?
Evaluation – Which model best meets the business objectives?
Deployment – How do stakeholders access the results?
Published in 1999 to standardize data mining processes across industries, it has since become the most common methodology for data mining, analytics, and data science projects.

Data science teams that combine a loose implementation of CRISP-DM with overarching team-based agile project management approaches will likely see the best results.

SOURCES:
https://www.datascience-pm.com/crisp-dm-2/


https://web.archive.org/web/20220401041957/https://www.the-modeling-agency.com/crisp-dm.pdf


https://www.oreilly.com/library/view/machine-learning-pocket/9781492047537/
Book sections
Introduction
Overview of the Machine Learning Processing
Classification Walkthrough: Titanic Dataset
Missing Data
Cleaning Data
Exploring
Preprocess Data
Feature Selection
Imbalanced Classes
Classification
Model Selection
Metrics and Classification Evaluation
Explaining Models
Regression
Metrics and Regression Evaluation
Explaining Regression Models
Dimensionality Reduction
Clustering
Pipelines


In [1]:
# create requirements.txt with required python code packages
%%writefile requirements.txt
matplotlib
pandas
sklearn 
yellowbrick
xlrd

Writing requirements.txt


In [ ]:
# install 
!pip install -r requirements.txt
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

In [3]:
# import tools and libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (ensemble, preprocessing,tree)
from sklearn.metrics import (auc, confusion_matrix, roc_auc_score, roc_curve)
from sklearn.model_selection import (train_test_split, StratifiedKFold)
from yellowbrick.classifier import (ConfusionMatrix, ROCAUC)
from yellowbrick.model_selection import (LearningCurve)
import ydata_profiling
from ydata_profiling import ProfileReport
import IPython

In [6]:
# GATHER DATA - load an excel file
url = ("https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic3.xls")
df = pd.read_excel(url) # create a data frame (matrix of data) with column labels (as opposed to numpy array)
orig_df = df 

### PREPARE AND CLEAN DATA 
# 1. strategy to determine data readiness and quality for use
# 2. hanlde the  issues found, once quality or readiness is determined CLEAN DATA 

In [ ]:

# inspect #rows columns & summary stats
df.shape 
df.describe().iloc[:, :2]
df.dtypes

In [ ]:

#note on axis 0 - along the index - apply the operation on said axis 


# a. analyze or spot check data for format we can use  ie min max, 
df.isnull().sum() # count of missing data in each column 
df.isnull().sum(axis=1).loc[:10] #
df.isnull().mean() * 100 # get the percentage of null values
# c. check for type of values like integer or float, NA or missing data, outliers, leaky features (info that are not available for new data yet or cheating by giving data to the model)
# d. use count for checking for missing data - only includes values not NaN - SME can help to know what to do 
# e. box plot or histogram outliers  

# extended note on data types : look for pandas friendly data types to store data--> int64 float64 datetime64[ns] object (combination of string and other). 
# pandas will try to coerce data and fall back to object, convert date and strings to numeric or feature engineer one. (cardinality h/l and dummy columns?)
mask = df.isnull().any(axis=1)
mask.head() 
df[mask].body.head()


In [ ]:
# f. profile data 
ydata_profiling.ProfileReport(df)
profile = ProfileReport(df, title="data set", html={'style' : {'full_width':True}})
profile.to_file(output_file="file.html")

# View HTML profile report
IPython.display.HTML(filename='/content/file.html')

In [ ]:

# a. impute 
# b. to repace or deal with missing or bad data: Standardize, Normalize, Outliers(check min/max), Missing Data(check for NaN), 
# c. Use S the most common to  
# d. drop them -- One possible strategy is to drop columns with no variance or signal or perfect or very high positive or negative correlation
# e. use pandas to create dummy columns with all 0s 


# conclusions for dropping: 
# age (interpolate values), 
# cabin
# boat
# body missing data in many rows and leaks data (gives survived or deceased)
df = df.drop(
    columns=[
        "name",
        "ticket",
        "home.dest",
        "boat",
        "body",  
        "cabin"
    ]) 

df.sex.value_counts(dropna=False)
df.embarked.value_counts(dropna=False)

In [ ]:
## CREATE FEATURES 
name = df.name
name.head(3)


df = pd.get_dummies(df)
df.columns

df = df.drop(columns="sex_male")
# OR df = pd.get_dummies(df, drop_first=True)
df.columns

y = df.survived
X = df.drop(columns="survived")

In [ ]:
## SAMPLE DATA - pull out 30% 
# X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=42)
#not working NameError: name 'model_selection' is not defined

# Impute missing values for age on training set 